In [33]:
import pandas as pd
import numpy as np
df = pd.read_csv('C:/Users/tanth/Downloads/Hanoi Hourly.csv')
df.head()

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,Hanoi,2015-01-01T00:00:00,15.0,15.0,13.0,87.84,0.0,0,NaN,0.0,...,1023.0,0.0,3.5,0.0,0.0,0.0,NaN,Clear,clear-night,"48820099999,VVNB"
1,Hanoi,2015-01-01T01:00:00,15.8,15.8,13.0,83.33,0.0,0,NaN,0.0,...,1022.6,0.0,8.5,0.0,0.0,0.0,NaN,Clear,clear-night,"48820099999,48825099999,48831099999,VVNB"
2,Hanoi,2015-01-01T02:00:00,13.5,13.5,12.0,90.65,0.0,0,NaN,0.0,...,1022.0,0.0,3.0,0.0,0.0,0.0,NaN,Clear,clear-night,"48820099999,VVNB"
3,Hanoi,2015-01-01T03:00:00,13.0,13.0,12.0,93.65,0.0,0,NaN,0.0,...,1022.0,0.0,2.0,0.0,0.0,0.0,NaN,Clear,clear-night,VVNB
4,Hanoi,2015-01-01T04:00:00,14.7,14.7,12.5,86.57,0.0,0,NaN,0.0,...,1022.4,0.0,8.2,0.0,0.0,0.0,NaN,Clear,clear-night,"48820099999,48825099999,VVNB"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94248 entries, 0 to 94247
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              94248 non-null  object 
 1   datetime          94248 non-null  object 
 2   temp              94248 non-null  float64
 3   feelslike         94248 non-null  float64
 4   dew               94248 non-null  float64
 5   humidity          94248 non-null  float64
 6   precip            94209 non-null  float64
 7   precipprob        94248 non-null  int64  
 8   preciptype        12055 non-null  object 
 9   snow              94206 non-null  float64
 10  snowdepth         94206 non-null  float64
 11  windgust          94199 non-null  float64
 12  windspeed         94247 non-null  float64
 13  winddir           94246 non-null  float64
 14  sealevelpressure  94248 non-null  float64
 15  cloudcover        94248 non-null  float64
 16  visibility        94070 non-null  float6

In [ ]:
#df = df.drop(columns=["snow", "snowdepth", "name", "preciptype", "visibility", "severerisk", "icon", "stations"])
#df["mising_any"] = df.isna().any(axis=1)
#lấy giá trị true trong missing_any
#df[df["mising_any"]]

In [34]:
# Chia train-test split theo thời gian, đảm bảo cột `datetime` đã là datetime và dữ liệu được sắp xếp
# temp là biến mục tiêu

# Parse và sắp xếp theo datetime (idempotent nếu đã chạy trước đó)
df["datetime"] = pd.to_datetime(df["datetime"]) 
df = df.sort_values("datetime").reset_index(drop=True)

train_size = int(len(df) * 0.8)
# Tập huấn luyện kết thúc trước 90 ngày (720 giờ) so với tập kiểm tra
train_data = df.iloc[:train_size - 720]
# Tập kiểm tra bắt đầu từ train_size đến hết
test_data = df.iloc[train_size:]

print(f"Kích thước tập huấn luyện: {train_data.shape}")
print(f"Kích thước tập kiểm tra: {test_data.shape}")
print(f"Khoảng thời gian tập huấn luyện: {train_data['datetime'].min()} đến {train_data['datetime'].max()}")
print(f"Khoảng thời gian tập kiểm tra: {test_data['datetime'].min()} đến {test_data['datetime'].max()}")

# Xây dựng features/target
feature_cols = df.columns.drop(['temp', 'datetime'])  # Loại bỏ cột temp khỏi features
X_train = train_data[feature_cols].copy()
y_train = train_data['temp']
X_test = test_data[feature_cols].copy()
y_test = test_data['temp']

# Đặt DatetimeIndex cho các dataframe con để interpolate(method='time') hoạt động
X_train.index = pd.DatetimeIndex(train_data['datetime'].values)
X_test.index = pd.DatetimeIndex(test_data['datetime'].values)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

Kích thước tập huấn luyện: (74678, 24)
Kích thước tập kiểm tra: (18850, 24)
Khoảng thời gian tập huấn luyện: 2015-01-01 00:00:00 đến 2023-07-09 13:00:00
Khoảng thời gian tập kiểm tra: 2023-08-08 14:00:00 đến 2025-10-01 23:00:00
X_train shape: (74678, 22), y_train shape: (74678,)
X_test shape: (18850, 22), y_test shape: (18850,)


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Đảm bảo dữ liệu có cột datetime và cột mục tiêu 'temp'
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.sort_values("datetime").reset_index(drop=True)

# Xác định features và target
target = "temp"
X = df.drop(columns=[target, "datetime"])
y = df[target]

# Khởi tạo TimeSeriesSplit (expanding window)
tscv = TimeSeriesSplit(n_splits=6, test_size=8760, gap=720)  # test mỗi lần 1 năm

# Duyệt qua từng fold
for fold, (train_idx, test_idx) in enumerate(tscv.split(X), 1):
    X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Set a DatetimeIndex on the split fragments so interpolate(method='time') works
    # Use the original dataframe's datetime values for the corresponding indices
    X_train.index = pd.DatetimeIndex(df['datetime'].iloc[train_idx].values)
    X_test.index = pd.DatetimeIndex(df['datetime'].iloc[test_idx].values)

    print(f"\nFold {fold}:")
    print(f"Train range: {df['datetime'].iloc[train_idx[0]].date()} → {df['datetime'].iloc[train_idx[-1]].date()} "
          f"({len(train_idx)} samples)")
    print(f"Test range:  {df['datetime'].iloc[test_idx[0]].date()} → {df['datetime'].iloc[test_idx[-1]].date()} "
          f"({len(test_idx)} samples)")

### Basic processing

In [35]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline

# Bước 1: Loại bỏ cột số có phương sai = 0
class VarianceThresholdSelector(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0):
        self.threshold = threshold
        self.selector = VarianceThreshold(threshold=self.threshold)
    
    def fit(self, X, y=None):
        numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
        self.numeric_cols = numeric_cols
        self.selector.fit(X[numeric_cols])
        # Lưu lại cột số được giữ lại
        self.retained_cols = numeric_cols[self.selector.get_support()]
        # Lưu lại cột số bị loại
        self.dropped_cols = [col for col in numeric_cols if col not in self.retained_cols]
        print("Các cột bị loại vì phương sai = 0:", self.dropped_cols)
        return self
    
    def transform(self, X):
        cols_to_keep = list(self.retained_cols) + list(X.columns.difference(self.numeric_cols))
        return X[cols_to_keep]

# Bước 2: Loại bỏ cột hằng số và dòng trùng lặp
class ConstantRemover(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Xác định cột hằng số
        self.constant_cols = [col for col in X.columns if X[col].nunique() == 1]
        print("Các cột bị loại vì chỉ có 1 giá trị:", self.constant_cols)
        return self
    
    def transform(self, X):
        # Bỏ cột hằng số
        X_cleaned = X.drop(columns=self.constant_cols, errors="ignore")
        return X_cleaned

    
# Tạo pipeline
cleaning_pipeline = Pipeline(steps=[
    ("var_thresh", VarianceThresholdSelector(threshold=0)),
    ("const", ConstantRemover())
])

In [36]:
from sklearn.base import BaseEstimator, TransformerMixin
# IterativeImputer is experimental — enable it explicitly
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

class PrecipImputer(BaseEstimator, TransformerMixin):
    def __init__(self, precip_col="precip", precipprob_col="precipprob", short_limit=6):
        self.precip_col = precip_col
        self.precipprob_col = precipprob_col
        self.short_limit = short_limit
        self.imputer = None

    def fit(self, X, y=None):
        df = X.copy()
        if self.precipprob_col in df.columns:
            mask_zero = df[self.precip_col].isna() & (df[self.precipprob_col] == 0)
            df.loc[mask_zero, self.precip_col] = 0.0
        df[self.precip_col] = df[self.precip_col].interpolate(method="time", limit=self.short_limit)
        cols = [c for c in [self.precip_col, "cloudcover", "humidity", "temp", self.precipprob_col, "windspeed"] if c in df.columns]
        if len(cols) > 1:
            self.imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=50, max_depth=5, n_jobs=-1, random_state=0), max_iter=5)
            self.imputer.fit(df[cols])
        return self

    def transform(self, X):
        df = X.copy()
        if self.precipprob_col in df.columns:
            mask_zero = df[self.precip_col].isna() & (df[self.precipprob_col] == 0)
            df.loc[mask_zero, self.precip_col] = 0.0
        df[self.precip_col] = df[self.precip_col].interpolate(method="time", limit=self.short_limit)
        if self.imputer:
            cols = self.imputer.feature_names_in_
            df[cols] = self.imputer.transform(df[cols])
        return df

In [37]:
from sklearn.impute import KNNImputer
import numpy as np

class WinddirImputer(BaseEstimator, TransformerMixin):
    def __init__(self, winddir_col="winddir", windspeed_col="windspeed", short_limit=6, knn_neighbors=5):
        self.winddir_col = winddir_col
        self.windspeed_col = windspeed_col
        self.short_limit = short_limit
        self.knn_neighbors = knn_neighbors
        self.imputer = None

    def fit(self, X, y=None):
        df = X.copy()
        wd_rad = np.deg2rad(df[self.winddir_col])
        df["wd_sin"] = np.sin(wd_rad)
        df["wd_cos"] = np.cos(wd_rad)
        df["wd_sin"] = df["wd_sin"].interpolate(method="time", limit=self.short_limit)
        df["wd_cos"] = df["wd_cos"].interpolate(method="time", limit=self.short_limit)
        cols = ["wd_sin", "wd_cos"]
        if self.windspeed_col in df.columns:
            cols.append(self.windspeed_col)
        self.imputer = KNNImputer(n_neighbors=self.knn_neighbors)
        self.imputer.fit(df[cols])
        return self

    def transform(self, X):
        df = X.copy()
        wd_rad = np.deg2rad(df[self.winddir_col])
        df["wd_sin"] = np.sin(wd_rad)
        df["wd_cos"] = np.cos(wd_rad)
        df["wd_sin"] = df["wd_sin"].interpolate(method="time", limit=self.short_limit)
        df["wd_cos"] = df["wd_cos"].interpolate(method="time", limit=self.short_limit)
        cols = ["wd_sin", "wd_cos"]
        if self.windspeed_col in df.columns:
            cols.append(self.windspeed_col)
        df[cols] = self.imputer.transform(df[cols])
        df[self.winddir_col] = (np.rad2deg(np.arctan2(df["wd_sin"], df["wd_cos"])) + 360) % 360
        return df

In [38]:
class SolarradiationImputer(BaseEstimator, TransformerMixin):
    def __init__(self, sol_col="solarradiation", lat=21.0278, short_limit=6):
        self.sol_col = sol_col
        self.lat = lat
        self.short_limit = short_limit
        self.imputer = None

    def solar_elevation(self, index):
        doy = index.dayofyear.values
        hour = index.hour + index.minute / 60.0
        decl = 23.44 * np.sin(2 * np.pi * (284 + doy) / 365)
        decl_rad = np.deg2rad(decl)
        lat_rad = np.deg2rad(self.lat)
        hour_angle = np.deg2rad((hour - 12) * 15)
        elev = np.arcsin(np.sin(lat_rad) * np.sin(decl_rad) + np.cos(lat_rad) * np.cos(decl_rad) * np.cos(hour_angle))
        return np.rad2deg(elev)

    def fit(self, X, y=None):
        df = X.copy()
        df["solar_elev"] = self.solar_elevation(df.index)
        df.loc[df["solar_elev"] <= 0, self.sol_col] = 0.0
        df.loc[df["solar_elev"] > 0, self.sol_col] = df.loc[df["solar_elev"] > 0, self.sol_col].interpolate(method="time", limit=self.short_limit)
        cols = [c for c in [self.sol_col, "cloudcover", "uvindex", "solarenergy", "temp", "humidity"] if c in df.columns]
        if len(cols) > 1:
            self.imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=50, max_depth=5, n_jobs=-1, random_state=0), max_iter=5)
            self.imputer.fit(df[cols])
        return self

    def transform(self, X):
        df = X.copy()
        df["solar_elev"] = self.solar_elevation(df.index)
        df.loc[df["solar_elev"] <= 0, self.sol_col] = 0.0
        df.loc[df["solar_elev"] > 0, self.sol_col] = df.loc[df["solar_elev"] > 0, self.sol_col].interpolate(method="time", limit=self.short_limit)
        if self.imputer:
            cols = self.imputer.feature_names_in_
            df[cols] = self.imputer.transform(df[cols])
        return df

In [39]:
from sklearn.pipeline import Pipeline

impute_pipeline = Pipeline([
    ("impute_precip", PrecipImputer()),
    ("impute_winddir", WinddirImputer()),
    ("impute_solar", SolarradiationImputer())
])

### Xử lí outlier

In [16]:
df.describe()

,datetime,temp,feelslike,dew,humidity,precip,precipprob,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk
count,94248,94248.000000,94248.000000,94248.000000,94248.000000,94209.000000,94248.000000,94206.0,94206.0,94199.000000,94247.000000,94246.000000,94248.000000,94248.000000,94070.000000,94212.000000,94212.000000,94212.000000,32645.000000
mean,2020-05-17 11:30:00,24.896258,27.393700,20.485905,78.270513,0.217643,8.245268,0.0,0.0,17.380778,9.318116,142.613537,1011.082052,64.712259,8.599728,155.174609,0.558068,1.545100,17.062031
min,2015-01-01 00:00:00,5.900000,2.100000,-8.000000,17.590000,0.000000,0.000000,0.0,0.0,1.100000,0.000000,0.000000,985.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,2017-09-08 17:45:00,21.000000,21.000000,17.000000,67.360000,0.000000,0.000000,0.0,0.0,11.500000,5.600000,65.000000,1005.000000,36.100000,7.000000,0.000000,0.000000,0.000000,10.000000
50%,2020-05-17 11:30:00,25.900000,25.900000,22.400000,81.880000,0.000000,0.000000,0.0,0.0,16.600000,8.500000,115.000000,1010.900000,81.900000,10.000000,5.800000,0.000000,0.000000,10.000000
75%,2023-01-24 05:15:00,29.000000,34.200000,25.000000,91.340000,0.000000,0.000000,0.0,0.0,22.000000,12.100000,185.000000,1016.800000,89.900000,10.000000,248.125000,0.900000,2.000000,15.000000
max,2025-10-01 23:00:00,41.900000,52.500000,31.000000,100.000000,186.240000,100.000000,0.0,0.0,118.800000,53.600000,360.000000,1041.000000,100.000000,24.100000,1011.900000,3.600000,10.000000,100.000000
std,NaN,5.613481,8.500585,5.931406,15.748502,2.647468,27.505461,0.0,0.0,7.716193,4.824510,104.672241,7.315970,34.604449,2.848757,235.708445,0.849264,2.370613,15.945075


In [ ]:
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Custom transformer: log1p cho các cột outlier
log_transformer = FunctionTransformer(np.log1p, validate=True)

# Các nhóm cột
outlier_cols = ["precip", "solarradiation"]
scale_cols = ["humidity", "cloudcover", "sealevelpressure"]

# Pipeline cho các cột outlier
log_outlier_pipeline = Pipeline(steps=[
    ('log_trans', log_transformer), # Biến đổi log(x+1)
    ('scaler', StandardScaler())    # Chuẩn hóa
])

# ColumnTransformer để xử lý theo nhóm
preprocessor = ColumnTransformer(
    transformers=[
        ("log_outliers", log_outlier_pipeline, outlier_cols),          # log1p + scale cho các cột lệch phải
        ("scale_num", StandardScaler(), scale_cols),                # scale cho các biến số
        ("encode_cat", OneHotEncoder(handle_unknown='ignore'), ["conditions"])   # OneHot cho categorical
    ],
    remainder="drop"  
)

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, FunctionTransformer

# Pipeline: log1p + scale
log_scale_pipeline = Pipeline([
    ("log1p", FunctionTransformer(np.log1p, validate=True))
])

# Pipeline: scale thường
scale_pipeline = Pipeline([
    ("scale", StandardScaler())
])

selected_bases = [
    "humidity", "precip", "winddir", "solarradiation",
    "sealevelpressure", "cloudcover", "wind_u", "wind_v", 
    "solar_elev", "hour", "doy"
]
# ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        # 1. log1p + scale cho các biến gốc lệch phải
        ("log_scale", log_scale_pipeline, lambda X: [
            col for col in X.columns
            if col in ["precip", "solarradiation"]
            and X[col].dtype.kind in "fi"
        ]),

        # 2. scale cho tất cả các biến số còn lại, trừ các biến đã one-hot
        ("scale_num", scale_pipeline, lambda X: [
            col for col in X.columns
            if (X[col].dtype.kind in "fi" and
            any(base in col for base in selected_bases))
])
    ],
    remainder="drop"
)

### FE

In [45]:
def solar_elevation(index, lat):
    doy = index.dayofyear.values
    hour = index.hour + index.minute / 60.0
    decl = 23.44 * np.sin(2 * np.pi * (284 + doy) / 365.25)
    decl_rad = np.deg2rad(decl)
    lat_rad = np.deg2rad(lat)
    hour_angle = np.deg2rad((hour - 12) * 15)
    elev = np.arcsin(np.sin(lat_rad) * np.sin(decl_rad) +
                     np.cos(lat_rad) * np.cos(decl_rad) * np.cos(hour_angle))
    return np.rad2deg(elev)


def add_hanoi_hourly_fe(
    df,
    datetime_col="datetime",
    lat=21.0278,
    vars_to_process=None,
    lags_short_hours=(1, 3, 6),
    lags_long_hours=(12, 24),
    same_hour_lags=(24,),
    rolling_windows_hours=(3, 6, 12, 24, 72)
):
    out = df.copy()

    # Ensure datetime index
    if datetime_col in out.columns:
        out[datetime_col] = pd.to_datetime(out[datetime_col])
        out = out.set_index(datetime_col)
    else:
        out.index = pd.to_datetime(out.index)
    out = out.sort_index()
    out = out.asfreq("h")

    # Variables to process
    if vars_to_process is None:
        vars_to_process = ["humidity", "precip", "cloudcover", "solarradiation"]

    # Time features
    out["hour"] = out.index.hour
    out["hour_sin"] = np.sin(2 * np.pi * out["hour"] / 24.0)
    out["hour_cos"] = np.cos(2 * np.pi * out["hour"] / 24.0)

    out["dayofyear"] = out.index.dayofyear
    out["doy_sin"] = np.sin(2 * np.pi * out["dayofyear"] / 365.25)
    out["doy_cos"] = np.cos(2 * np.pi * out["dayofyear"] / 365.25)


    out["month"] = out.index.month
    # One-hot encoding mùa
    m = out["month"]
    out["season_spring"] = m.isin([3, 4]).astype(int)
    out["season_summer"] = m.isin([5, 6, 7, 8]).astype(int)
    out["season_autumn"] = m.isin([9, 10, 11]).astype(int)
    out["season_winter"] = m.isin([12, 1, 2]).astype(int)

    # One-hot encode the 'icon' categorical column (group rare categories into 'other')
    if "icon" in out.columns:
        cond_ser = out["icon"].fillna("missing").astype(str)
        cond_dummies = pd.get_dummies(cond_ser, prefix="icon")
        out = pd.concat([out, cond_dummies], axis=1)
        out = out.drop(columns=["icon"], errors="ignore")

    # Solar elevation
    out["solar_elev"] = solar_elevation(out.index, lat)

    # Rolling cho solar_elev
    for w in rolling_windows_hours:
        out[f"solar_elev_roll_mean_{w}h"] = out["solar_elev"].shift(1).rolling(window=w, min_periods=1).mean()

    # Wind features
    if "winddir" in out.columns:
        deg = pd.to_numeric(out["winddir"], errors="coerce")
        deg_mod = deg % 360
        rad = np.deg2rad(deg_mod)
        out["winddir_sin"] = np.sin(rad)
        out["winddir_cos"] = np.cos(rad)
        out["winddir_rad"] = rad

        if "windspeed" in out.columns:
            spd = pd.to_numeric(out["windspeed"], errors="coerce").fillna(0.0)
        else:
            spd = pd.Series(0.0, index=out.index)

        out["wind_u"] = -spd * np.sin(rad)
        out["wind_v"] = -spd * np.cos(rad)

        out["monsoon_NE"] = ((deg_mod >= 20) & (deg_mod < 80)).astype(int)
        out["monsoon_SW"] = ((deg_mod >= 200) & (deg_mod < 260)).astype(int)
        out["monsoon_Other"] = (~(out["monsoon_NE"].astype(bool) | out["monsoon_SW"].astype(bool))).astype(int)
    else:
        out["winddir_sin"] = 0.0
        out["winddir_cos"] = 0.0
        out["wind_u"] = 0.0
        out["wind_v"] = 0.0
        out["monsoon_NE"] = 0
        out["monsoon_SW"] = 0
        out["monsoon_Other"] = 1

    # Rolling cho wind_u, wind_v
    for w in rolling_windows_hours:
        out[f"wind_u_roll_mean_{w}h"] = out["wind_u"].shift(1).rolling(window=w, min_periods=1).mean()
        out[f"wind_v_roll_mean_{w}h"] = out["wind_v"].shift(1).rolling(window=w, min_periods=1).mean()

    # Precip summaries
    if "precip" in out.columns:
        out["precip_flag"] = (out["precip"].fillna(0) > 0).astype(int)
        out["precip_24h_sum"] = out["precip"].shift(1).rolling(window=24, min_periods=1).sum()
        mask_rain = out["precip"].fillna(0) > 0
        idx_series = pd.Series(out.index, index=out.index)
        last_rain_time = idx_series.where(mask_rain).ffill()
        hours_since_rain = (out.index.to_series() - last_rain_time).dt.total_seconds() / 3600.0
        out["hours_since_rain"] = hours_since_rain.fillna(9999.0)
    else:
        out["precip_flag"] = 0
        out["precip_24h_sum"] = 0.0
        out["hours_since_rain"] = 9999.0

    # Rolling cho precip_flag
    for w in rolling_windows_hours:
        out[f"precip_flag_roll_sum_{w}h"] = out["precip_flag"].shift(1).rolling(window=w, min_periods=1).sum()

    # Lag features cho các biến chính
    # ensure same_hour_lags is iterable
    if isinstance(same_hour_lags, int):
        same_hour_iter = (same_hour_lags,)
    else:
        same_hour_iter = same_hour_lags

    lags = sorted(set(list(lags_short_hours) + list(lags_long_hours) + list(same_hour_iter)))
    for col in vars_to_process:
        if col not in out.columns:
            continue
        for lag in lags:
            out[f"{col}_lag_{lag}h"] = out[col].shift(lag)

    # Rolling mean cho các biến chính
    for col in vars_to_process:
        if col not in out.columns:
            continue
        for w in rolling_windows_hours:
            base = out[col].shift(1).rolling(window=w, min_periods=1)
            out[f"{col}_roll_mean_{w}h"] = base.mean()

    # Cleanup: luôn loại bỏ các cột phụ không cần thiết
    drop_cols = ["winddir_rad", "dayofyear", "hour", "month"]
    out = out.drop(columns=[c for c in drop_cols if c in out.columns], errors="ignore")
    return out


In [46]:
from sklearn.base import BaseEstimator, TransformerMixin

class HanoiHourlyFE(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        datetime_col="datetime",
        lat=21.0278,
        vars_to_process=None,
        lags_short_hours=(1, 3, 6),
        lags_long_hours=(12, 24),
        same_hour_lags=(24,),
        rolling_windows_hours=(3, 6, 12, 24, 72)    
    ):
        self.datetime_col = datetime_col
        self.lat = lat
        self.vars_to_process = vars_to_process
        self.lags_short_hours = lags_short_hours
        self.lags_long_hours = lags_long_hours
        self.same_hour_lags = same_hour_lags
        self.rolling_windows_hours = rolling_windows_hours

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return add_hanoi_hourly_fe(
            df=X.copy(),
            datetime_col=self.datetime_col,
            lat=self.lat,
            vars_to_process=self.vars_to_process,
            lags_short_hours=self.lags_short_hours,
            lags_long_hours=self.lags_long_hours,
            same_hour_lags=self.same_hour_lags,
            rolling_windows_hours=self.rolling_windows_hours        
    )

In [47]:
# Inspect FE output WITHOUT including the target column to avoid leakage
x_fe = HanoiHourlyFE().fit_transform(X_train)
x_fe.columns.tolist()

['name',
 'feelslike',
 'dew',
 'humidity',
 'precip',
 'precipprob',
 'preciptype',
 'snow',
 'snowdepth',
 'windgust',
 'windspeed',
 'winddir',
 'sealevelpressure',
 'cloudcover',
 'visibility',
 'solarradiation',
 'solarenergy',
 'uvindex',
 'severerisk',
 'conditions',
 'stations',
 'hour_sin',
 'hour_cos',
 'doy_sin',
 'doy_cos',
 'season_spring',
 'season_summer',
 'season_autumn',
 'season_winter',
 'icon_clear-day',
 'icon_clear-night',
 'icon_cloudy',
 'icon_fog',
 'icon_partly-cloudy-day',
 'icon_partly-cloudy-night',
 'icon_rain',
 'icon_wind',
 'solar_elev',
 'solar_elev_roll_mean_3h',
 'solar_elev_roll_mean_6h',
 'solar_elev_roll_mean_12h',
 'solar_elev_roll_mean_24h',
 'solar_elev_roll_mean_72h',
 'winddir_sin',
 'winddir_cos',
 'wind_u',
 'wind_v',
 'monsoon_NE',
 'monsoon_SW',
 'monsoon_Other',
 'wind_u_roll_mean_3h',
 'wind_v_roll_mean_3h',
 'wind_u_roll_mean_6h',
 'wind_v_roll_mean_6h',
 'wind_u_roll_mean_12h',
 'wind_v_roll_mean_12h',
 'wind_u_roll_mean_24h',
 'wind

In [48]:
# Pipeline hoàn chỉnh
full_pipeline = Pipeline(steps=[
    ("cleaning", cleaning_pipeline),
    ("fe", HanoiHourlyFE()),  
    ("imputation", impute_pipeline),
    ("preprocess", preprocessor),
    ("model", RandomForestRegressor(max_depth=10))  # Sử dụng Random Forest cho bài toán hồi quy
])

full_pipeline.fit(X_train, y_train)


Các cột bị loại vì phương sai = 0: ['snow', 'snowdepth']
Các cột bị loại vì chỉ có 1 giá trị: ['name', 'preciptype']


Pipeline(steps=[('cleaning',
                 Pipeline(steps=[('var_thresh', VarianceThresholdSelector()),
                                 ('const', ConstantRemover())])),
                ('fe', HanoiHourlyFE()),
                ('imputation',
                 Pipeline(steps=[('impute_precip', PrecipImputer()),
                                 ('impute_winddir', WinddirImputer()),
                                 ('impute_solar', SolarradiationImputer())])),
                ('preprocess',
                 ColumnTransformer(transformers=[('log_scale',
                                                  Pipeline(steps=[('log1p',
                                                                   FunctionTransformer(func=<ufunc 'log1p'>,
                                                                                       validate=True))]),
                                                  <function <lambda> at 0x000001E60173C860>),
                                                 ('scale_num',
                                                  Pipeline(steps=[('scale',
                                                                   StandardScaler())]),
                                                  <function <lambda> at 0x000001E60173CC20>)])),
                ('model', RandomForestRegressor(max_depth=10))])

In [49]:
from sklearn.metrics import mean_squared_error, r2_score

# Prepare input for prediction: use original test_data so we keep datetime and temp
X_test_input = test_data[feature_cols].copy()
# ensure DatetimeIndex for transformers that expect it
X_test_input.index = pd.DatetimeIndex(test_data['datetime'].values)

# Run prediction (this will apply cleaning -> fe -> imputation -> preprocess inside the pipeline)
y_pred = full_pipeline.predict(X_test_input)

# Compute metrics (y_test aligned to original test_data rows may contain more rows than predictions
# if cleaning removed rows; so align y_true to the indices used by predictions)
# Recreate the transformed test-frame used before the model to find the index alignment
clean = full_pipeline.named_steps.get('cleaning')
fe = full_pipeline.named_steps.get('fe')

Xc = clean.transform(X_test_input) if clean is not None else X_test_input
test_fe_model = fe.transform(Xc) if fe is not None else Xc

# Get index used for predictions
pred_idx = test_fe_model.index if hasattr(test_fe_model, 'index') else None

# Align true temperatures to prediction rows using datetime index from original test_data
temp_series = test_data.set_index(pd.DatetimeIndex(test_data['datetime'].values))['temp']

y_true_aligned = temp_series.loc[pred_idx].values if pred_idx is not None else y_test.values

mse  = mean_squared_error(y_true_aligned, y_pred)
rmse = np.sqrt(mse)
r2   = r2_score(y_true_aligned, y_pred)

print(f"Test RMSE: {rmse:.3f}")
print(f"Test R^2 : {r2:.3f}")

# Build results DataFrame with datetime, true temp and predicted temp
results = pd.DataFrame({
    'datetime': pd.DatetimeIndex(pred_idx),
    'temp_true': y_true_aligned,
    'temp_pred': y_pred
})

print(results.head(10))

c:\Users\tanth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tanth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tanth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Test RMSE: 1.528
Test R^2 : 0.924
             datetime  temp_true  temp_pred
0 2023-08-08 14:00:00       33.0  34.550479
1 2023-08-08 15:00:00       31.0  31.054814
2 2023-08-08 16:00:00       32.5  33.335135
3 2023-08-08 17:00:00       28.0  28.094008
4 2023-08-08 18:00:00       27.0  26.591006
5 2023-08-08 19:00:00       29.3  28.738951
6 2023-08-08 20:00:00       27.0  26.503849
7 2023-08-08 21:00:00       28.0  26.628181
8 2023-08-08 22:00:00       28.8  28.724462
9 2023-08-08 23:00:00       28.0  28.003144


In [50]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Prepare input for training predictions (keep datetime for alignment)
X_train_input = train_data[feature_cols].copy()
X_train_input.index = pd.DatetimeIndex(train_data['datetime'].values)

# Predict on training input (applies cleaning -> fe -> imputation -> preprocess -> model)
y_train_pred = full_pipeline.predict(X_train_input)

# Recreate transformed train-frame used before the model to find index alignment
clean = full_pipeline.named_steps.get('cleaning')
fe = full_pipeline.named_steps.get('fe')

Xc_train = clean.transform(X_train_input) if clean is not None else X_train_input
train_fe_model = fe.transform(Xc_train) if fe is not None else Xc_train

# Get index used for predictions
pred_idx_train = train_fe_model.index if hasattr(train_fe_model, 'index') else None

# Align true temperatures to prediction rows using datetime index from original train_data
temp_series_train = train_data.set_index(pd.DatetimeIndex(train_data['datetime'].values))['temp']

y_train_aligned = temp_series_train.loc[pred_idx_train].values if pred_idx_train is not None else y_train.values

# Compute metrics
mse_train = mean_squared_error(y_train_aligned, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train_aligned, y_train_pred)

print(f"Train RMSE: {rmse_train:.3f}")
print(f"Train R^2 : {r2_train:.3f}")

# Quick peek at first rows
train_results = pd.DataFrame({'datetime': pd.DatetimeIndex(pred_idx_train), 'temp_true': y_train_aligned, 'temp_pred': y_train_pred})
print(train_results.head(10))

c:\Users\tanth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tanth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tanth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Train RMSE: 1.099
Train R^2 : 0.962
             datetime  temp_true  temp_pred
0 2015-01-01 00:00:00       15.0  15.339716
1 2015-01-01 01:00:00       15.8  14.882606
2 2015-01-01 02:00:00       13.5  13.730995
3 2015-01-01 03:00:00       13.0  13.835051
4 2015-01-01 04:00:00       14.7  13.972696
5 2015-01-01 05:00:00       12.1  13.726835
6 2015-01-01 06:00:00       11.4  13.700045
7 2015-01-01 07:00:00       13.3  13.655189
8 2015-01-01 08:00:00       13.5  14.072658
9 2015-01-01 09:00:00       16.5  17.211459


In [51]:
#dự đoán nhiệt độ giờ tiếp theo
next_hour_features = X_test_input.iloc[[-1]]  # Lấy dòng cuối cùng của tập kiểm tra
next_hour_pred = full_pipeline.predict(next_hour_features)
#ghi rõ datetime của giờ tiếp theo
next_hour_datetime = next_hour_features.index[0] + pd.Timedelta(hours=1)
print(f"Dự đoán nhiệt độ cho giờ tiếp theo ({next_hour_datetime}): {next_hour_pred[0]:.2f}°C")

Dự đoán nhiệt độ cho giờ tiếp theo (2025-10-02 00:00:00): 25.08°C


c:\Users\tanth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\tanth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
